In [5]:
! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn
! pip install optuna

In [6]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/")

In [7]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://reddit-reccomender-bucket/11', creation_time=1763396487435, experiment_id='11', last_update_time=1763396487435, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [8]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from sklearn.metrics import accuracy_score,classification_report,f1_score,make_scorer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [9]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [10]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for KNN regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

## Step 2: Train-test split BEFORE sampling

X_train_raw,X_test_raw,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
)

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000

vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply Random Undersampler to train only
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, y_train)



#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparison")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors',3,30)
    p = trial.suggest_categorical('p', [1, 2])

    model = KNeighborsClassifier(n_neighbors=n_neighbors,p=p)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test_vec)

    return accuracy_score(y_test,preds)


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train_res, X_test_vec, y_train_res, y_test)

# Run the experiment for KNN
run_optuna_experiment()




[I 2025-11-18 13:13:29,771] A new study created in memory with name: no-name-a68bd076-7a7c-452f-8c74-2a359411ca31
[I 2025-11-18 13:13:36,757] Trial 0 finished with value: 0.41347333969725897 and parameters: {'n_neighbors': 30, 'p': 1}. Best is trial 0 with value: 0.41347333969725897.
[I 2025-11-18 13:13:40,215] Trial 1 finished with value: 0.42970135006136645 and parameters: {'n_neighbors': 9, 'p': 2}. Best is trial 1 with value: 0.42970135006136645.
[I 2025-11-18 13:13:43,382] Trial 2 finished with value: 0.3996999863630165 and parameters: {'n_neighbors': 24, 'p': 2}. Best is trial 1 with value: 0.42970135006136645.
[I 2025-11-18 13:13:46,778] Trial 3 finished with value: 0.45738442656484385 and parameters: {'n_neighbors': 13, 'p': 1}. Best is trial 3 with value: 0.45738442656484385.
[I 2025-11-18 13:13:49,148] Trial 4 finished with value: 0.3863357425337515 and parameters: {'n_neighbors': 29, 'p': 2}. Best is trial 3 with value: 0.45738442656484385.
[I 2025-11-18 13:13:52,177] Trial 

🏃 View run KNN_Undersampling_BOW_Trigram at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11/runs/43fa756c18db48d8bde727959b33bd9a
🧪 View experiment at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11
